In [1]:
from ipycanvas import Canvas, hold_canvas

In [2]:
from ipywidgets import widgets, HBox, VBox, Layout

In [3]:
import math

In [4]:
robot_canvas_width = 500
robot_canvas_height = 300
    
robot_canvas = Canvas(width=robot_canvas_width, height=robot_canvas_height)

In [5]:
def draw_robot_canvas(canvas, rf):
    with hold_canvas():
        canvas.clear()
        canvas.reset_transform()

        # The origin of the canvas is the top which is upside down so we will
        # move it to the bottom left corner and invert the y-axis
        canvas.translate(0, canvas.height)
        canvas.scale(1.0, -1.0)

        canvas.line_width = 15
        canvas.line_cap = 'round'
        
        # Draw vertical line for robot base
        canvas.begin_path()
        canvas.stroke_style = "red"
        canvas.stroke_lines([rf.b0, rf.j0])
        canvas.stroke()
        
        # Draw first link (j0-j1)
        canvas.begin_path()
        canvas.stroke_style = "green"
        canvas.stroke_lines([rf.j0, rf.j1])
        canvas.stroke()

        # Draw second link (j1-j2)
        canvas.begin_path()
        canvas.stroke_style = "blue"
        canvas.stroke_lines([rf.j1, rf.j2])
        canvas.stroke()


In [6]:
class robot_configuration:
    base_x = 150
    base_height = 10
    
    l1 = 120
    j0_degrees = 90
    
    l2 = 120
    j1_degrees = -90
        
rc = robot_configuration()

In [13]:
class robot_figure:
    
    def __init__(self, robot_configuration):
        self.rc = robot_configuration
        self.b0 = [0,0]
        self.j0 = [0,0]
        self.j1 = [0,0]
        self.j2 = [0,0]
        self.d2r = math.pi/180 # convert degrees to radians
    
    def update_locations(self):
        #Draw the base first
        self.b0 = [self.rc.base_x, 20]

        self.j0 = [self.b0[0], self.b0[1]+self.rc.base_height]

        # Add the first link to the base
        self.j1[0] = self.j0[0] + self.rc.l1 * math.cos(self.rc.j0_degrees*self.d2r)
        self.j1[1] = self.j0[1] + self.rc.l1 * math.sin(self.rc.j0_degrees*self.d2r)
        
        # Add the second link to the first link
        self.j2[0] = self.j1[0] + self.rc.l2 * math.cos((self.rc.j0_degrees+self.rc.j1_degrees)*self.d2r)
        self.j2[1] = self.j1[1] + self.rc.l2 * math.sin((self.rc.j0_degrees+self.rc.j1_degrees)*self.d2r)

rf = robot_figure(rc)


In [14]:
pos_x_widget = widgets.FloatSlider(
    value=90,
    min=0,
    max=360.0,
    step=0.1,
    description='J0 degrees:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=Layout( width='700px' ),
)

def on_value_change_x(change):
    rc.j0_degrees = (change['new'])
    rf.update_locations()
    draw_robot_canvas(robot_canvas, rf)

pos_x_widget.observe(on_value_change_x, names='value')

pos_y_widget = widgets.FloatSlider(
    value=90,
    min=0,
    max=360.0,
    step=0.1,
    description='J1 Degrees:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    readout_format='.1f',
    layout=Layout( height='300px' ),
)

def on_value_change_y(change):
    rc.j1_degrees= (change['new'])
    rf.update_locations()
    draw_robot_canvas(robot_canvas, rf)
    
pos_y_widget.observe(on_value_change_y, names='value')


In [15]:
cb = HBox([pos_y_widget, robot_canvas])
tb = HBox([pos_x_widget])
VBox([tb,cb])